In [2]:
import numpy as np
import scipy as scp
import pandas as pd

In [3]:
# package edu.michigan.gevirtz.data;

# import java.util.Arrays;

class DataVector:
    def __init__(self, values):
        self.label = -1
        self.values = values

    def setValues(self, values):
        self.values = values
    
    def getValues(self):
        return self.values
    
    def setLabel(self, label):
        self.label = label

    def GetLabel(self):
        return self.label


In [ ]:
# package edu.michigan.gevirtz.optimization;

# import org.jblas.DoubleMatrix;

# public interface Minimizer {
    
#     public DoubleMatrix Minimize(OptimizationProblem problem, DoubleMatrix x0);

# }

In [ ]:
# package edu.michigan.gevirtz.optimization;

# import org.jblas.DoubleMatrix;

class OptimizationProblem:
    pass
#     //Returns parameters to be optimized
#     public double Function(DoubleMatrix x);
#     public DoubleMatrix Gradient(DoubleMatrix x);


In [ ]:
# package edu.michigan.gevirtz.optimization;

# import org.jblas.DoubleMatrix;
# import org.jblas.Solve;

class BFGSOptimizer:
    def __init__(self):
        self.problem = None # OptimizationProblem
        self.alpha = 0.1
        #  number of steps in to search while doing line search for new parameter vector

        self.k = 100

    #Setters
    """
    Sets the largest (in magnitude) alpha to be used in updating the current parameter vector 
    @param alpha
    """
    def SetMaximumAlpha(self, alpha):
        self.alpha = alpha

    def SetK(self, k):
        self.k = k


    def GetDirection(self, B, x):
        ***grad = self.problem.Gradient(x).mul(-1.0)
        ***return Solve.solve(B, grad)



    def PerformLineSearch(self, x, direction):
        currentAlpha = 0
        ***currentMinimum = Double.MAX_VALUE
        step = self.alpha / self.k
        ***for(int i = 1; i <= k; i++ ):
            ***new_x = x.add(direction.mul(step * i))
            ***val = problem.Function(new_x)
            if val < currentMinimum:
                currentAlpha = val
                currentAlpha = step * i

        return currentAlpha


    ***@Override
    def Minimize(self, problem, x0):

        self.problem = problem

        ***N = x0.rows

        ***B = DoubleMatrix.eye(N);
        ***x = x0.dup();

#        while(problem.Gradient(x).norm2() >= .05){
        while problem.Function(x) >= 0.05:
            direction = self.GetDirection(B, x);
            ***norm = direction.norm2()
            ***direction.muli(1/norm);

            alpha = self.PerformLineSearch(x, direction)

            ***s = direction.mul(alpha)
            ***x_new = x.add( s )


            ***y = problem.Gradient(x_new).sub(  problem.Gradient(x) )

            ***denominator1 = y.transpose().mmul(s).get(0)

            ***denomintaor2 = s.transpose().mmul(B.mmul(s)).get(0)

            ***B_new = B.add(y.mmul(y.transpose()).mul( 1 / denominator1 ))
            ***B_new.subi(B.mmul(s.mmul(s.transpose().mmul(B))).mul( 1 / denomintaor2))

            B = B_new
            x = x_new

            print("Grad: "+problem.Gradient(x).norm2())
            print("Func: "+problem.Function(x))

        return x


In [ ]:
# package edu.michigan.gevirtz.latent;

# import java.awt.image.SampleModel;
# import java.io.BufferedReader;
# import java.io.BufferedWriter;
# import java.io.FileReader;
# import java.io.FileWriter;
# import java.io.IOException;
# import java.util.ArrayList;
# import java.util.HashMap;
# import java.util.List;
# import java.util.Map;
# import java.util.Random;

# import javax.xml.crypto.Data;

# import org.jblas.DoubleMatrix;
# import org.jblas.MatrixFunctions;
# import org.jblas.Singular;
# import org.jblas.Solve;

# import edu.michigan.gevirtz.data.DataVector;
# import edu.michigan.gevirtz.optimization.BFGSOptimizer;
# import edu.michigan.gevirtz.optimization.OptimizationProblem;

class GTM:

    def __init__(self):
        self.data = null # List<DataVector>
        #  holds the training data in jBLAS vectors
        self.dataVectors = null # List<DoubleMatrix>
        self.T = None     #  Holds the data in a matrix;

        self.L = 0 # Dimensionality of latent space
        self.M = 0 # Number of basis functions
        self.K = 0 # Number of points in latent spaces
        self.D = 0 # Dimensionality of observable space
        self.N = 0

        self.K_per_latent_dimension = 0
        self.M_per_latent_dimension = 0

        self.sigma = 0 # Width of gaussian basis functions

        self.phiMatrix = null # Matrix containing evaluation of
                                                # all M
        self.W = null # Matrix defining y(x->t)
        self.beta = 0   # variance of distribution of t
        # basis functions on all K latent space
        # sample points

        self.basisFunctionCenters = null    # Centers for the basis
                                                            # functions

        self.latentSpaceSamplePoints = null   # Center for each x_i in the latent space



    """
    @param data
    Data to train the latent model on
    @param L
    Dimensionality of the latent space 
    """

    def SubtractMean(data):
        totals = new double[data.get(0).getValues().length]
        for(int i = 0; i < data.get(0).getValues().length; i++):
            totals[i] = 0
        for(DataVector v : data):
            for(int i = 0; i < v.getValues().length;i++):
                totals[i] += v.getValues()[i]
        for(int i = 0; i < data.get(0).getValues().length; i++):
            totals[i] /= data.size()
        for(int i = 0; i < data.size(); i++):
            double [] values = data.get(i).getValues()
            for(int j = 0; j < values.length; j++)
                values[j] -= totals[j]

#  data.remove(i);
#  data.add(i, new DataVector(values));
            data.get(i).SetValues(values)

    def GTM(data, L):

        self.SubtractMean(data);
        self.data = data;

        self.L = L;

        self.N = len(data)

        self.dataVectors = new ArrayList<DoubleMatrix>()
        for(DataVector dv : data):
            self.dataVectors.add( new DoubleMatrix(dv.getValues()) )

        self.T = dataVectors.get(0).transpose();
        for(int i = 1; i < dataVectors.size(); i++):
            self.T = DoubleMatrix.concatVertically(self.T, dataVectors.get(i).transpose())

        print("dataVectors.size(): "+dataVectors.size())
        print("T: "+T.rows+"x"+T.columns)
        self.D = data.get(0).getValues().length


    # Setters
    def setSigma(sigma):
        self.sigma = sigma


    def setM(M_per_latent_dimension):

        self.M_per_latent_dimension = M_per_latent_dimension
        self.M = (int) Math.pow(M_per_latent_dimension, L)


    def setK(K_per_latent_dimension):

        self.K_per_latent_dimension = K_per_latent_dimension
        self.K = (int) Math.pow(K_per_latent_dimension, L)


    def CalculatePhiMatrix():

        phiMatrix = new DoubleMatrix(K,M);//CalculatePhiVector(latentSpaceSamplePoints[0]).transpose();
        for(int i = 0; i < K; i++):
            phiMatrix.putRow(i,CalculatePhiVector(latentSpaceSamplePoints[i]).transpose());

            
    def ProjectDataToLatentSpace():

        return ProjectDataVectorToLatentSpace(self.data)
            
    def ProjectDataVectorToLatentSpace(vectors):
        print("ProjectDataVectorToLatentSpace")
        result = new ArrayList<DataVector>()
        print("\tCalculating R...")
        R = CalculateR(W, beta, vectors)
        print("done")
        N = len(vectors)
        print("\tAdding contributions...")
        for(int i = 0; i < N; i++):
            vector = new DoubleMatrix(L)
            for(int j = 0 ; j < K; j++):
                vector.addi(latentSpaceSamplePoints[j].mul(R.get(j,i)) )
            dv = new DataVector(vector.toArray())
            dv.SetLabel(vectors.get(i).GetLabel())
            result.add(dv)
        
        print("done")
        return result
    
    def ProjectToDataSpace(t):
        return W.mmul(CalculatePhiVector(t))
    

    def CalculatePhiVector(x):
        phi = new DoubleMatrix(M)
        for (int i = 0; i < M; i++):
            phi.put(i, calculateGaussian(x, basisFunctionCenters[i], sigma, L))
        return phi
    

    # Getters

    # Internal

    """
     * Calls GenerateBasisFunctionCenters, setups up W matrix, calls
     * CalculatePhi
    """

    def InitializeModel():
        print("Initializing model...")

        if (sigma == 0):
            throw new RuntimeException(
                    "Attempted to initialize model withouth setting basis function width, sigma");

        print("\tGenerating basis function centers...")
        GenerateBasisFunctionCenters()
        print("done")
        print("\tGenerating latent space smaple points...")
        GenerateLatentSpaceSamplePointCenters()
        print("done")
        print("\tCalculating phi matrix...")
        CalculatePhiMatrix()
        print("done")
        
        #Find the initial W parameters
        W = DoubleMatrix.rand(D, M)
        
        paramInitializer = new ParameterInitializer(this, 2000)
        optimizer = new BFGSOptimizer()
        W_new = optimizer.Minimize(paramInitializer, new DoubleMatrix(W.toArray()))
        #  Put the column-vector representation of W_new into W 
        for(int i = 0; i < D; i++):
            for(int j = 0; j < M ; j ++):
                W.put(i ,j , W_new.get(i * M + j))

        
        #  find initial beta parameter
        pca = new PCA(data.subList(0, 1000))
        
        beta_inv = pca.GetLargestEigenvalue()
        
        self.beta = 1.0 / beta_inv
        print("done")


    def CalculateDataSpaceDistribution(t, W, beta):
        result = 0
        for(int x_i = 0 ; x_i < K; x_i++):
            result += CalculateDataSpaceDistribution(t, x_i, W, beta)
        
        return result / K
        
    def CalculateDataSpaceDistribution(t, x, W, beta):
        return calculateGaussian(t, W.mmul(CalculatePhiVector(x)), beta, self.D)
    
    def CalculateDataSpaceDistribution(t, x_i, W, beta):
        return calculateGaussian(t, W.mmul(phiMatrix.getRow(x_i).transpose()), beta, self.D) #  Changed from calculating phi(x_i) every thime
    
    def CalculateDataSpaceDistribution(t):
        return CalculateDataSpaceDistribution(t, self.W, self.beta)
    
    def CalculateR(W, beta):
        return CalculateR(W, beta, self.data)
    
    def CalculateR(W, beta, data):
        N = data.size()
        R = new DoubleMatrix(K, N)
        
        denominators = new double[N]
        
        for(int i = 0; i < N; i++):
            vector = dataVectors.get(i)    #  Changed from creating a new thing based on the data
            denominators[i] = CalculateDataSpaceDistribution(vector, W, beta) * K
        
        for(int i = 0; i < K; i++):
            for(int j = 0; j < N; j++):
#                vector = new DoubleMatrix(data.get(j).getValues())
                vector = dataVectors.get(j)    #  Changed from creating a new thing based on the data
                R.put(i,j, CalculateDataSpaceDistribution(vector, latentSpaceSamplePoints[i], W, beta) / denominators[j])
        
        return R
    
    def CalculateG(DoubleMatrix W, double beta):
        R = CalculateR(W, beta)
        G = new DoubleMatrix(K,K)

        for(int i = 0; i < K; i++):
            total = 0.0
            for(int j = 0; j < N; j++):
                total += R.get(i, j)
            G.put(i,i,total)
        
        return G
    
    """
     * Sets the lattice of basis functions
    """
    def GenerateBasisFunctionCenters():
        if (M == 0):
            throw new RuntimeException(
                    "Attempted to generate basis function centers without first setting M");
        if (L == 0):
            throw new RuntimeException(
                    "Attempted to generate basis function centers without first setting L");

        currentIndices = new int[L]
        for (int i = 0; i < L; i++):
            currentIndices[i] = 0
        spacing = 2.0 / (M_per_latent_dimension - 1)

        basisFunctionCenters = new DoubleMatrix[M]

        for (int i = 0; i < M; i++):
            #  update the indices
            for (int j = 0; j < L; j++):
                num = (int) Math.pow(M_per_latent_dimension, j)
                currentIndices[j] = (i / num) % M_per_latent_dimension
            
            center = new DoubleMatrix(L)
            #  loop through each dimension, set the position of this center for
            #  each dimension based on the current values of "currentIndex"
            for (int j = 0; j < L; j++):
                center.put(j, spacing * currentIndices[j] - 1)

#            System.out.println("Basis function center: "+center)
            basisFunctionCenters[i] = center
            

    def GenerateLatentSpaceSamplePointCenters():
        if (K == 0):
            throw new RuntimeException(
                    "Attempted to generate latent space sample point centers without first setting K");
        if (L == 0):
            throw new RuntimeException(
                    "Attempted to generate latent space sample point centers without first setting L");

        currentIndices = new int[L]
        for (int i = 0; i < L; i++):
            currentIndices[i] = 0
        spacing = 2.0 / (K_per_latent_dimension - 1)

        latentSpaceSamplePoints = new DoubleMatrix[K]

        for (int i = 0; i < K; i++):
            #  update the indices
            for (int j = 0; j < L; j++):
                num = (int) Math.pow(K_per_latent_dimension, j)
                currentIndices[j] = (i / num) % K_per_latent_dimension
            
            center = new DoubleMatrix(L)
            # loop through each dimension, set the position of this center for
            # each dimension based on the current values of "currentIndex"
            for (int j = 0; j < L; j++):
                center.put(j, spacing * currentIndices[j] - 1)
#            System.out.println("latent space sample point: "+center)
            latentSpaceSamplePoints[i] = center
    

    def calculateGaussian(point, center, width, D):
        if (point.columns != 1):
            throw new RuntimeException("Point not a column vector")
        if (center.columns != 1):
            throw new RuntimeException("Center not a column vector!")
        if (point.columns != center.columns):
            throw new RuntimeException("Dimensions are not equal")

        return Math.pow(width / (2 * 3.14159), D / 2.0)
                * Math.exp(-1*center.squaredDistance(point))


    def SVDInvert(DoubleMatrix matrix):
        svd = Singular.fullSVD(matrix)
        
        U = svd[0]
        W = svd[1]
        V = svd[2]
        
        W_inverse = new DoubleMatrix(W.rows, W.rows)
        for(int i = 0; i < W.rows; i++):
            W_inverse.put(i,i,1.0 / W.get(i,0))
        return V.mmul(W_inverse.mmul(U.transpose()))

    
    def PerformEMStep():
        print("Performing EM Step")
        
        print("\tCalculating R using old parameters...")
        R_old = CalculateR(W, beta)
        print("done")
        print("\tCalculating A using old parameters...")
        A = phiMatrix.transpose().mmul(CalculateG(W, beta).mmul(phiMatrix))
        print("done")
        print("\tCalculating B using old parameters...")
        B = phiMatrix.transpose().mmul(R_old.mmul(T))
        print("done")

        print("\tInverting to get new W...")
        W =  SVDInvert(A).mmul(B).transpose()
        print("done")

        print("\tUpdating beta...")
        beta_inv = 0
        for(int i = 0; i < N; i++):
            for(int j = 0; j < K; j++):
                phi = phiMatrix.getRow(j).transpose()
                beta_inv += R_old.get(j,i) * W.mmul(phi).squaredDistance(dataVectors.get(i))
            
        beta_inv /= (D * N)
        
        beta = 1.0 / beta_inv
        print("done")
    
    def WriteDataVectors(dataVectors, filename):
        try:
            writer = new BufferedWriter( new FileWriter(filename) )
            for(DataVector vector : dataVectors):
                writer.write(vector.GetLabel() + " ")
                values = vector.getValues()
                for(int i = 0; i < values.length; i++):
                    writer.write(values[i]+" ")
                writer.write("\n")
                    
            writer.close()
            
        catch IOException e:
            print(e)

    
    
    def WritePosteriorDistribution(data, filename):
        R = CalculateR(W, beta, data)
        values = new HashMap<Integer, Double>()
        writer = null
        try:
            writer = new BufferedWriter(new FileWriter(filename))
        catch IOException e:
            print(e)
            return
        
        for(int i = 0; i < K; i++):
            values.clear()
            for(int n = 0; n < data.size(); n++):
                label = data.get(n).GetLabel()
                if(!values.containsKey(label)):
                    values.put(label, 0.0)
                
                value = values.get(label)
                value += R.get(i,n)
                
                values.put(label, value )
                

            for(int label : values.keySet()):
                center = latentSpaceSamplePoints[i]
                try:    
                    writer.write(label+" ")
                    writer.write(values.get(label) / data.size()+" ")
                    for(int l = 0; l < L; l++):
                        writer.write(center.get(l)+" ")
                    writer.write("\n")
                    
                catch IOException e:
                    print(e)
                    return
         
        try:
            writer.close()
        catch IOException e:
            print(e)

    
    class ParameterInitializer implements OptimizationProblem:

        """
         * 
         * @param gtm
         * @param n
         *            number of the subset of training vectors to use for
         *            parameter initialization. Not all training vectors are
         *            needed; using a subsample increases speed
        """
        def __init__(self, GTM gtm, n):

            
            self.gtm = None
            self.n = 0
            self.inversePCA = None
            self.transformedLatentSamplePoints = None

            
            
            self.gtm = gtm;
            self.n = n;
            
            PCA pca = new PCA(gtm.data.subList(0, n));

            inversePCA = pca.getTransform(L).transpose();
            transformedLatentSamplePoints = new ArrayList<DoubleMatrix>();
            for(DoubleMatrix point : gtm.latentSpaceSamplePoints)
                transformedLatentSamplePoints.add(inversePCA.mmul(point));
            
//            NormalizeTransformedLatentSamplePoints();
            
        }
        
        private void NormalizeTransformedLatentSamplePoints(){
            double [] max = new double[gtm.D];
            double [] min = new double[gtm.D];
            double [] mean = new double[gtm.D];
            for(int i = 0; i < max.length; i++){
                max[i] = Double.MIN_VALUE;
                min[i] = Double.MAX_VALUE;
            }
            for(DoubleMatrix point : transformedLatentSamplePoints)
                for(int i = 0; i < gtm.D; i++){
                    mean[i] += point.get(i);
                    if(Math.abs(point.get(i)) > max[i])
                        max[i] = Math.abs(point.get(i));
                    if(Math.abs(point.get(i)) < min[i])
                        min[i] = Math.abs(point.get(i));
                }
            double [] norms = new double[gtm.D];
            for(int i = 0; i < gtm.D; i++){
                norms[i] = max[i] - min [i];
                mean[i] /= transformedLatentSamplePoints.size();
            }
            
            
            for(DoubleMatrix point : transformedLatentSamplePoints)
                for(int i = 0; i < gtm.D; i++){
                    double value = point.get(i);
                    point.put(i, (value - mean[i])/ (norms[i] / 2) + mean[i]) ;
                }
                    
                
        }

        /**
         * x contains the elements of W
         */
        @Override
        public double Function(DoubleMatrix x) {

            DoubleMatrix W = new DoubleMatrix(D,M);
            for(int i = 0; i < D; i++)
                for(int j = 0; j < M; j++)
                    W.put(i, j, x.get(i * M + j));
            
            double total = 0.0;
            //Loop over all latent space sample points
            for(int i = 0; i < K; i++){
                DoubleMatrix transformedLatentSpaceSamplePoint = transformedLatentSamplePoints.get(i);
                total += W.mmul(gtm.phiMatrix.getRow(i).transpose()).distance2(transformedLatentSpaceSamplePoint );
            }
            return .5 * total / gtm.K; // / gtm.M;
        }

        @Override
        public DoubleMatrix Gradient(DoubleMatrix x) {
            DoubleMatrix W = new DoubleMatrix(D,M);
            for(int i = 0; i < D; i++)
                for(int j = 0; j < M; j++)
                    W.put(i, j, x.get(i * M + j));
            DoubleMatrix gradient = new DoubleMatrix(D*M);
            DoubleMatrix vector = new DoubleMatrix(D*M);
            for(int i = 0; i < K; i++){
                DoubleMatrix phi = gtm.phiMatrix.getRow(i).transpose();
                DoubleMatrix Ux = transformedLatentSamplePoints.get(i);
                DoubleMatrix Wphi = W.mmul(phi);
                double denominator = Wphi.distance2(Ux);
                //D x M
                for(int j = 0 ; j < D * M; j++){
                    int a = j / M;
                    int b = j % M;
                    
                    vector.put(j, .5*phi.get(b)*( Wphi.get(a) - Ux.get(a) ) / denominator);
                    
                }
                gradient.addi(vector);
            }
            
            return gradient;
        }

    }
    
    
    private static void test1(){
        List<DataVector> data = new ArrayList<DataVector>();
        Random rand = new Random();
        int N = 10000;
        for (int i = 0; i < N; i++){
            double x = rand.nextDouble();
            double y = x + ( rand.nextDouble() - .5 ) / .5 * .01;
            double [] values = {x,y};
            data.add(new DataVector(values));
            
        }
        //Initialize the GTM
        GTM gtm = new GTM(data, 1);
        System.out.println("df");
        gtm.setSigma(.01);
        gtm.setK(200);
        gtm.setM(2);
        gtm.InitializeModel();

        List<DoubleMatrix> testingData = new ArrayList<DoubleMatrix>();
        for (int i = 0; i < 1000; i++){
            double x = rand.nextDouble();
            double y = x + ( rand.nextDouble() - .5 ) / .5 * .01;
            double [] values = {x};
            DoubleMatrix latentPoint = new DoubleMatrix(values);
            
            testingData.add(latentPoint);
        }
        
        
        //Create a PCA transform from latent to data space:
        PCA pca = new PCA(data.subList(0, 1000));
        DoubleMatrix inversePCA = pca.getTransform(2).transpose();

        //First test PCA.  Write stuff to "pca.txt"
        try{
            BufferedWriter pcaWriter = new BufferedWriter(new FileWriter("pca.txt"));

            for(int i = 0; i < 1000; i++){
                DoubleMatrix dataSpaceProjectionPCA = inversePCA.mmul(testingData.get(i));
                for(int j = 0; j < dataSpaceProjectionPCA.rows; j++)
                    pcaWriter.write(dataSpaceProjectionPCA.get(j)+" ");
                pcaWriter.write('\n');
                
        }
            pcaWriter.close();
            BufferedWriter gtmWriter = new BufferedWriter(new FileWriter("gtm.txt"));
            for(int i = 0; i < 1000; i++){
                DoubleMatrix dataSpaceProjectionGTM = gtm.ProjectToDataSpace(testingData.get(i));
                for(int j = 0; j < dataSpaceProjectionGTM.rows; j++)
                    gtmWriter.write(dataSpaceProjectionGTM.get(j)+" ");
                gtmWriter.write('\n');
            }
            gtmWriter.close();
        }catch(IOException e){
            e.printStackTrace();
        }
        
        //Now perform a single step of the EM aglorithm
        
        gtm.PerformEMStep();
        
    }
    private static void test2(String [] argv){
        
        
        String dataFile = argv[0];
        List<DataVector> data = new ArrayList<DataVector>();
        //Read data in
        try{
            BufferedReader reader = new BufferedReader(new FileReader(dataFile));
            String line = "";
            while((line = reader.readLine()) != null){
                String [] elements = line.split("\\s+");
                double [] values = new double[elements.length - 1];
                for(int i = 1; i < elements.length; i++)
                    values[i-1] = Double.valueOf(elements[i]);
                DataVector vector = new DataVector(values);
                vector.SetLabel(Integer.valueOf(elements[0]));
                data.add(vector);
            }
                
            reader.close();
        }catch(IOException e){
            e.printStackTrace();
        }
        
        GTM gtm = new GTM(data,2);
        gtm.setSigma(Double.valueOf(argv[1])); //.5
        gtm.setK(Integer.valueOf(argv[2])); //50
        gtm.setM(Integer.valueOf(argv[3]));    //4
        gtm.InitializeModel();
        System.out.print("Writing pca transformed latent space sample points...");
        PCA pca = new PCA(data);
        DoubleMatrix inversePCA = pca.getTransform(2).transpose();
        List<DataVector> transformedSamplePoints = new ArrayList<DataVector>();
        for(DoubleMatrix vector : gtm.latentSpaceSamplePoints){
            DataVector dv = new DataVector( inversePCA.mmul(vector).toArray() );
            
            transformedSamplePoints.add(dv);
        }
        String baseName = "gtm_output/";
        
        gtm.WriteDataVectors(transformedSamplePoints, baseName+"transformed_sample_points.txt");
        System.out.println("done");
        int N = 1000;
        
        for(int i = 0; i < N; i++){
//            System.out.println(gtm.W);
            List<DataVector> projection = gtm.ProjectDataToLatentSpace();
            
            
            gtm.WriteDataVectors(projection, baseName+String.format("%03d", i)+".txt");
            gtm.WritePosteriorDistribution(projection, baseName+String.format("prior/%03d", i)+".txt");
            
            //Look at how latent sample points transform
            transformedSamplePoints.clear();
            System.out.print("Projecting latent space sample points to data space...");
            for(DoubleMatrix vector : gtm.latentSpaceSamplePoints){
                DoubleMatrix phi = gtm.CalculatePhiVector( vector );
                DataVector dv = new DataVector( gtm.W.mmul(phi).toArray() );
                
                transformedSamplePoints.add(dv);
            }
            System.out.println("done.");
            
            gtm.WriteDataVectors(transformedSamplePoints, baseName+"latent/"+String.format("%03d", i)+".txt");
            System.out.println("Performing EM step!");
            System.out.println(gtm.beta);
            gtm.PerformEMStep();
        }
        
        
    }
    public static void main(String [] argv){
        
        test2(argv);
        
        
    }

}

In [ ]:
package edu.michigan.gevirtz.latent;


import java.io.BufferedReader;
import java.io.FileReader;
import java.io.IOException;
import java.util.ArrayList;
import java.util.Collections;
import java.util.Comparator;
import java.util.List;

import org.jblas.ComplexDoubleMatrix;
import org.jblas.DoubleMatrix;
import org.jblas.Eigen;

import edu.michigan.gevirtz.data.DataVector;

public class PCA {


    private DoubleMatrix dataMatrix = null;

    private DoubleMatrix covarianceMatrix = null;
    
    private DoubleMatrix eigenVectors = null;
    
    private DoubleMatrix eigenValues = null;
    
    private void BuildCovarianceMatrix(){
        
        //dataMatrix.columns is the number of variables there are per data vector
        double [][] result = new double[dataMatrix.columns][dataMatrix.columns];
        
        int N = dataMatrix.rows;
        
        for(int i = 0; i < dataMatrix.columns; i++)
            for(int j = 0; j < dataMatrix.columns; j++){
                
                double total = 0;
                for(int k = 0; k < dataMatrix.rows; k++)
                    total += dataMatrix.get(k,i) * dataMatrix.get(k,j);  

                result[i][j] = total / ( N - 1);
                result[j][i] = total / ( N - 1);
                
            }
        
        covarianceMatrix = new DoubleMatrix(result);
        
        
    }
    
    
    public void CalculatePCA(){
        
        DoubleMatrix values = Eigen.eigenvectors(covarianceMatrix)[1].real();
        DoubleMatrix  vectors = Eigen.eigenvectors(covarianceMatrix)[0].real();
        

//        System.out.println(covarianceMatrix);
        List<EigenVectorEigenValuePair> pairs = new ArrayList<PCA.EigenVectorEigenValuePair>();
        for(int i = 0; i < vectors.columns; i++){
            pairs.add( new EigenVectorEigenValuePair(vectors.getColumn(i), values.get(i,i)));
//            System.out.println("Eigenvector: "+vectors.getColumn(i));
        }
        
        Collections.sort(pairs, pairs.get(0).comparator);
        
        //Create the column vector that will contain the eigenvalues
        eigenValues = new DoubleMatrix(pairs.size());
        //Fill the structures holding eigenvectors and eigenvalues
        int index = 0;
        for(EigenVectorEigenValuePair pair : pairs){
            
            //if eigenVectors is null, we need to add the first of the eigenvectors.
            if(eigenVectors == null){
                eigenVectors = pair.eigenvector.dup();
                continue;
            }
            //Otherwise, stick the next eigenvector at the end of the matrix
            eigenVectors = DoubleMatrix.concatHorizontally(eigenVectors, pair.eigenvector);
            eigenValues.put(index,0,pair.eigenvalue);
            index++;
            
            
            
        }
//        System.out.println("eigenValues: "+eigenValues);
        
    }
    
    public double GetLargestEigenvalue(){
        return eigenValues.get(0,0);
    }
        
    private void BuildDataMatrix(List<DataVector> data){

        int N = data.size();
        
        double [][] dataArray = new double[N][];
        
        for(int i = 0; i < data.size(); i++)
            dataArray[i] = data.get(i).getValues();
            
        
        dataMatrix = new DoubleMatrix(dataArray);
        
        //Subtract means
        DoubleMatrix columnMeans = dataMatrix.columnMeans();
        dataMatrix.subiRowVector(columnMeans);
    }
    
    public List<DataVector> transform(){
        
        List<DataVector> result = new ArrayList<DataVector>();
        DoubleMatrix product = eigenVectors.transpose().mmul( dataMatrix.transpose() );

        for(int i = 0; i < product.columns; i++){
            result.add( new DataVector(product.getColumn(i).toArray()) );
        }
        
        return result;
        
    }
    public List<DataVector> transform(List<DataVector> data, int L){
        
        List<DataVector> result = new ArrayList<DataVector>();
        
        DoubleMatrix transformation = getTransform(L);
        
        for(DataVector dataVector : data){
            DoubleMatrix vector = new DoubleMatrix(dataVector.getValues());
            DataVector newDataVector = new DataVector(transformation.mmul(vector).toArray());
            newDataVector.SetLabel( dataVector.GetLabel() );
            
            result.add(newDataVector);
            
        }
        
        return result;
        
    }
    
    public PCA(List<DataVector> data){
        
        BuildDataMatrix(data);
        
        BuildCovarianceMatrix();
        
        CalculatePCA();
        
    }
    
    public DoubleMatrix getTransform(int L){
        
        int rows = eigenVectors.rows;
        
        return eigenVectors.getRange(0,rows,0,L).transpose();
        
    }
    
    private class EigenVectorEigenValuePair{
        private Comparator<EigenVectorEigenValuePair> comparator = new Comparator<PCA.EigenVectorEigenValuePair>() {
            
            @Override
            public int compare(EigenVectorEigenValuePair o1,
                    EigenVectorEigenValuePair o2) {
                // TODO Auto-generated method stub
                double val = (o2.eigenvalue - o1.eigenvalue); 
                return (int)(val / Math.abs(val));
            }
        };
        
        public double eigenvalue = 0;
        public DoubleMatrix eigenvector = null;
        
        public EigenVectorEigenValuePair(DoubleMatrix eigenvector, double eigenvalue){
            self.eigenvalue = eigenvalue;
            self.eigenvector = eigenvector;
        }
        
    }
    
    
    //Simple test to be run from commandline.  Loads file supplied as arg[0], reads data vectors, spits out PCA-transformed vectors
    public static void main(String [] args){
        
        List<DataVector> data = new ArrayList<DataVector>();
        
        try{
            BufferedReader reader = new BufferedReader( new FileReader(args[0]) );
            String line;
            while((line = reader.readLine()) != null){
                String [] elements  = line.split("\\s+");
                double [] values = new double[elements.length];;
                for(int i = 1; i < elements.length; i++)
                    values[i] = Double.valueOf(elements[i]);
                DataVector dataVector = new DataVector( values ) ;
                dataVector.SetLabel(Integer.valueOf(elements[0]));
                data.add( dataVector );
            }
            
            reader.close();
        }catch(IOException e){
            e.printStackTrace();
            System.exit(1);
        }
        
        PCA pca = new PCA(data);
        for(DataVector dataVector : pca.transform(data, 2)){
            double [] values = dataVector.getValues();
            System.out.print(dataVector.GetLabel()+" ");
            for(double value : values)
                System.out.print( value +" ");
            System.out.println();
        }
            
        
    }
}